In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 22)
pd.set_option('display.max_columns', 100)
from datetime import datetime
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 9)
import matplotlib.style
matplotlib.style.use('ggplot')
#%pylab inline

In [2]:
def loadpanel(filename):
    df = pd.read_csv(filename, sep='\t', quoting=3, dtype=object)
    df.columns = [x.lower() for x in df.columns.str.strip('˙')]
    df.vl_ttl = df.vl_ttl.astype(str).apply(lambda x: x.replace(',','')).astype(float)
    df.qt_tran = df.qt_tran.astype(str).apply(lambda x: x.replace(',','')).astype(int)
    df['date'] = pd.to_datetime(df.ts.apply(lambda x: x[:-3]))
    df = df.groupby('date').sum()
    df.insert(1, 'vl_med', df.vl_ttl/df.qt_tran)
    df['ts'] = [int(time.mktime(time.strptime(str(x)[:-3], '%Y-%m-%d %H:%M'))) for x in df.index]
    df['cycle'] = df.ts.apply(lambda x: (x/60)%1440)
    return df

In [3]:
month = '07'
path = '/home/user'
device = loadpanel('{}/{}//atm_panel.tsv'.format(path, month))

In [4]:
me = device.count_tran['2016-07-14':'2016-07-20']
me = me[me < me.quantile(0.999)]#.plot()
#me.plot.bar()
me.plot()
#me.resample('H', how='sum').plot.bar()
plt.show()

In [63]:
home_user = 'user'
vd = device.count_tran['2016-07-03':'2016-07-09']
ho = vd.resample('H', how='sum').index.hour
#ocor = vd.resample('H', how='sum').values + np.random.normal(mu, sigma, 24).round()
ocor = list(map(lambda x: int(x) if not np.isnan(x) else 0, vd.resample('H', how='sum').values))
fr = list(zip(ho,ocor))
#fr = pd.DataFrame(list(zip(ho,ocor)), columns=['hour','ocor'])
#fr.ocor.plot()
#plt.show()
#print(fr)
with open('/home/{}/rpca/hour_week.data'.format(home_user), 'w') as fo:
    fo.write(','.join(map(str, ocor)))
len(fr)

168

In [68]:
home_user = 'user'

init = '2016-07-01'
end = '2016-07-31'
ind = pd.date_range(init, end, freq='T', closed='left')
#ind = pd.date_range(init, periods=1440*7, freq='T')
filler = pd.Series(np.nan, index=ind)
data = device.count_tran#[init:end]
#data = data[data < data.quantile(0.975)]
filler.update(data)
sm = filler.fillna(method='pad')#fillna(method='backfill')
sm = sm['2016-07-14':'2016-07-20']
repeat = 4
obs = np.tile(sm.values.astype(int), repeat)
obs = obs + 1#bug app
with open('/home/{}/rpca/dev_7d.tsd'.format(home_user), 'w') as fo:
    fo.write(','.join(map(str, obs)))

In [79]:
fr = {0: 39461, 1: 16330, 2: 7835, 3: 5422, 4: 6586, 5: 15475, 6: 47163, 7: 165291, 8: 366859, 9: 467935, 10: 480354, 11: 466204, 12: 355311, 13: 375890, 14: 421910,
      15: 409478, 16: 392508, 17: 335769, 18: 266695, 19: 238259, 20: 220426, 21: 193137, 22: 128794, 23: 72548}
mu, sigma = 5000, 3000
buf = []
for i in list(range(24))*30:
    #buf.append(fr[i] + round(np.random.normal(mu,sigma)))
    buf.append(fr[i] + np.random.uniform(-5422,5422))
    #buf.append(fr[i])
#plt.plot(buf)
#plt.show()
#with open('/home/user/rpca/start.data', 'w') as fo:
#    fo.write(','.join(map(str, buf)))

In [194]:
window = 5
cur = time.strptime('2016-07-08T17:59:59', '%Y-%m-%dT%H:%M:%S')
#stop = int(time.mktime(cur)) - (int(time.mktime(cur)) % 60)
stop = int(time.mktime(cur))
stop = stop - (stop % 60) + window
stop

window = 5*1000
stop - (stop % (60*1000)) + window

stop = 1468015150957/1000
datetime.fromtimestamp(stop)

datetime.datetime(2016, 7, 8, 18, 59, 10, 957000)

In [87]:
with open('/home/{}/rpca/dev_7d.tsd'.format(home_user)) as fi:
    buf = list(map(int, fi.read().split(',')))

In [83]:
np.median(buf),np.mean(buf),np.std(buf)

(4431.5, 8975.7513888888898, 9716.5998380194924)

In [84]:
pd.Series(buf).plot()
plt.show()

In [104]:
t0 = int(time.time())
t1 = t0 - dst
dst = 3600*3
#int(t1/60) % 1440*7
ind = int(t1/60) % (1440*7)
t1,ind, buf[ind]

(1471554521, 1268, 3086)